In [ ]:
import numpy as np
from kite import Scene
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import math
import utm
from osgeo import gdal

In [ ]:
inyml="elazig_asc_kite_scene"
outbase="elazig_asc"

In [ ]:
sc=Scene.load(inyml)
qt=sc.quadtree

In [ ]:
print("There are",qt.nleaves,"leaves")

In [ ]:
reflat=sc.frame.llLat
reflon=sc.frame.llLon

# we would also like the reference point in utm coordinates
refutm=utm.from_latlon(reflat,reflon)

print("reference point (lat-long):",reflat,"N",reflon,"E")

print("reference point (utm):",refutm[0],"E",refutm[1],"N")

In [ ]:
# xy positions of data points in utm kilometers
#xkm=(refutm[0]+qt.leaf_focal_points_meter[:,0])/1000
#ykm=(refutm[1]+qt.leaf_focal_points_meter[:,1])/1000

# xy positions of data points in long-lat
xyll=np.column_stack((qt.leaf_focal_points[:,0]+reflon,qt.leaf_focal_points[:,1]+reflat))

# los vector components using kite definiton of coordinates
losx=np.cos(qt.leaf_thetas)*-np.cos(qt.leaf_phis)
losy=np.cos(qt.leaf_thetas)*-np.sin(qt.leaf_phis)
losz=-np.sin(qt.leaf_thetas)

# id for each datapoint
dataptid=np.arange(1,losx.size+1)

In [ ]:
# let's convert our lat-long coordinates to UTM km
xym = utm.from_latlon(xyll[:,1],xyll[:,0]) 
xkm = xym[0]/1000
ykm = xym[1]/1000

In [ ]:
fig = plt.figure(figsize=(6,12))

ax = fig.add_subplot(111)
ax.set_xlabel("UTM_x (km)",fontsize=12)
ax.set_ylabel("UTM_y (km)",fontsize=12)
ax.grid(True,linestyle='-',color='0.75')

# scatter with colormap mapping to z value
ax.scatter(xkm,ykm,s=20,c=qt.leaf_means, marker = 'o', cmap = cm.jet );
ax.set_aspect('equal')

plt.show()

In [ ]:
# okinv format in km
okinv_data=np.column_stack((xkm,ykm,qt.leaf_means,losx,losy,losz,dataptid))
outfile=outbase+".okinv"
np.savetxt(outfile,okinv_data,fmt='%f %f %f %f %f %f %d')

# okinv format in lat-long 
okinv_data_ll=np.column_stack((xyll,qt.leaf_means,losx,losy,losz,dataptid))
outfile=outbase+"_ll"+".okinv"
np.savetxt(outfile,okinv_data_ll,fmt='%f %f %f %f %f %f %d')

In [ ]:
!ls

In [ ]:
!cat elazig_asc.okinv